# Making a Polling Application

Now, we add another application that asks users a question, processes a submission of a response, and displays the result of the poll.  While this is a very basic application, it demonstrates an important motivation for using Django, notably the ability to handle dynamic data.  To do so, we introduce an import object in an **HTML form** and also the **CRUD** paradigm to create, update, and destroy an object like a users poll response.  

## Creating the Pols app

In the main directory of your site, run the normal command to create a Django application

```UNIX
python manage.py startapp polls
```

### Create Basic View

We will start by creating a basic view to show we are at the index of the polls application.  In our `views.py` file, we enter the following.

```python
from django.http import HttpResponse


def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")
```

### Adding the URL

Now, we connect this view to a url.  We have to create a `urls.py` file.  After making the file, we enter the following.

```python
from django.urls import path

from . import views

urlpatterns = [
    path('', views.index, name='index'),
]
```

As per usual, we need to add our urls from the polls app to our larger site.  To do so we go to the main `urls.py` file, and enter the following:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('polls/', include('polls.urls')),
   ...
]
```

## Creating the Models

Now that we have the basic application up and running, let's go ahead and add the models for the questions.  In the `models.py` file, enter the following.

```python
from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
```

Also, let's make sure we add the polls application to our `settings.py` file.

### Add the Administration and Make Migrations

Now, we want to be able to use the admin site to add Questions and Choices.  To do so, we need to go to the `admin.py` file and enter the following.

```python
from django.contrib import admin

from .models import Question

admin.site.register(Question)
admin.site.register(Choice)
```

Also, be sure to create the tables in the database by creating and applying migrations.

```UNIX
python manage.py makemigrations
python manage.py migrate
```

### Adjust the Admin View

```python
from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
    #string method to return question text
    def __str__(self):
        return self.question_text


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
    
    def __str__(self):
        return self.choice_text
```

## Writing Views

Now that we have the application hooked up, and data ready to be made use of, we want to create our views.  In our `views.py` file, let's add to our initial view a *detail, results*, and *vote* view.  In our `views.py` file let's write the following:

```python
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)
```

### URLS

For each of our new views, we will create a link in our `urls.py` file as follows.

```python


urlpatterns = [
  ...
    # ex: /polls/5/
    path('<int:question_id>/', views.detail, name='detail'),
    # ex: /polls/5/results/
    path('<int:question_id>/results/', views.results, name='results'),
    # ex: /polls/5/vote/
    path('<int:question_id>/vote/', views.vote, name='vote'),
]
```

## View Extension

Now, we will adjust our index view to display the latest 5 questions.  To do so, we add a variable to our view that is the last five objects.  We do this with:

```python
latest_question_list = Question.objects.order_by('-pub_date')[:5]
```

then, we join these together and return the results to our `HttpResponse` method.

```python
output = ', '.join([q.question_text for q in latest_question_list])
    return HttpResponse(output)
```

Together, our updated `index` view function is as follows:

```python
from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    output = ', '.join([q.question_text for q in latest_question_list])
    return HttpResponse(output)
```

This is an important idea.  We are taking data from our database, doing something to it, and returning the result.  

### Template

As before, we will create a folder to hold our templates.  This time, we add the template for our index view within the polls application as follows

```UNIX
mkdir templates/polls/index.html
```

Inside of this, add the following code:

```HTML
{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}
```

Notice that we are asking the template to render an object called `latest_question_list`.  To do this, we need to adjust our view to include a **context** object that is a dictionary containing a list of objects.  To do so we add the following to our view:

```python
context = {
        'latest_question_list': latest_question_list,
    }
```



also, because we are rendering the response in a template, we ask our view to return an `HttpResponse` that renders the template as follows:

```python
return HttpResponse(template.render(context, request))
```

Together, this gives us the following view:

```python
from django.shortcuts import render

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}
    return render(request, 'polls/index.html', context)
```

### Raising 404 Errors

For our detail view, we want to provide for the situation where the object doesn't exist.  Adjust the view as follows:

```python
from django.shortcuts import get_object_or_404, render

from .models import Question
# ...
def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/detail.html', {'question': question})
```

### Detail Template

Our Question detail template will display a list of questions.  Let's create the following template in `templates/polls/detail.html`:

```HTML
<h1>{{ question.question_text }}</h1>
<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }}</li>
{% endfor %}
</ul>
```

### URLs

We can use the templating system to automate the urls in our templates.  For example, we can change

```HTML
<li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
```

to 

```HTML
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>
```

Now we can update our urls for the polling app as follows

```python
from django.urls import path

from . import views

app_name = 'polls'
urlpatterns = [
    path('', views.index, name='index'),
    path('<int:question_id>/', views.detail, name='detail'),
    path('<int:question_id>/results/', views.results, name='results'),
    path('<int:question_id>/vote/', views.vote, name='vote'),
]
```

We should change the url in our `polls/index.html` file also from 

```HTML
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>
```

to 

```HTML
<li><a href="{% url 'polls:detail' question.id %}">{{ question.question_text }}</a></li>
```